<a href="https://colab.research.google.com/github/Veena9977/Practice2/blob/main/Exit_coding_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
import re
nltk.download('punkt_tab')
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Dropout, Embedding, BatchNormalization, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

In [ ]:
data = pd.read_csv('/content/tweet_emotions.csv')

In [ ]:
print(data.shape)

(40000, 3)


In [ ]:
#Display the dtype information
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   40000 non-null  int64 
 1   sentiment  40000 non-null  object
 2   content    40000 non-null  object
dtypes: int64(1), object(2)
memory usage: 937.6+ KB
None


In [ ]:
data.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [ ]:
print(data['sentiment'].value_counts())

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64


Preprocessing

In [ ]:
#Checking missing values
data.isna().sum()

,0
tweet_id,0
sentiment,0
content,0


In [ ]:
#Checking null values
data.isnull().sum()

,0
tweet_id,0
sentiment,0
content,0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
print(nltk.data.find('tokenizers/punkt'))

/root/nltk_data/tokenizers/punkt


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
stopwords = set(nltk.corpus.stopwords.words('english'))

In [ ]:
from nltk.stem import PorterStemmer
porter_stemmer = PorterStemmer()

In [ ]:
#Using the function removes urls,special characters,stop words etc....It preserves the hash tags.And also  convert text to lower
#case and tokenize text and apply stemming
def clean_reviews(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove user mentions (@username)
    text = re.sub(r'@\S+', '', text)
    # Remove special characters except hashtags
    text = re.sub(r'[^A-Za-z#\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize the text (split into words)
    tokens = word_tokenize(text)

    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords]

    # Apply stemming
    stemmed_tokens = [porter_stemmer.stem(word) for word in tokens]

    # Apply lemmatization
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in stemmed_tokens]

    # Join words back into a string
    cleaned_text = ' '.join(lemmatized_tokens)

    return cleaned_text

In [ ]:
data['cleaned_content'] = data['content'].apply(clean_reviews)

In [ ]:
#Display the old and new cleaned tweets
print(data[['content', 'cleaned_content']].head())

                                             content  \
0  @tiffanylue i know  i was listenin to bad habi...   
1  Layin n bed with a headache  ughhhh...waitin o...   
2                Funeral ceremony...gloomy friday...   
3               wants to hang out with friends SOON!   
4  @dannycastillo We want to trade with someone w...   

                                     cleaned_content  
0  know listenin bad habit earlier start freakin ...  
1              layin n bed headach ughhhhwaitin call  
2                        funer ceremonygloomi friday  
3                              want hang friend soon  
4               want trade someon houston ticket one  


In [ ]:
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Adam

In [ ]:
tokenized_corpus = [word_tokenize(content) for content in data['cleaned_content']]

Bag of Words Model

In [ ]:
vectorizer = CountVectorizer(max_features=5000)  # Limit the number of features (words)
X_bow = vectorizer.fit_transform(data['cleaned_content']).toarray()

In [ ]:
print(X_bow.shape)

(40000, 5000)


In [ ]:
# Initialize the CountVectorizer with n-grams (1, 2)
cv = CountVectorizer(ngram_range=(1, 2))

# Apply fit_transform to the cleaned tweets
X = cv.fit_transform(data['cleaned_content'])

# Get the labels (y)
y = data['sentiment']

In [ ]:
data['sentiment']

,sentiment
0,empty
1,sadness
2,sadness
3,enthusiasm
4,neutral
...,...
39995,neutral
39996,love
39997,love
39998,happiness


In [ ]:
feature_names = cv.get_feature_names_out()
print("Feature Names (Unigrams):", feature_names[:20])

Feature Names (Unigrams): ['aa' 'aa that' 'aa work' 'aaa' 'aaa new' 'aaaa' 'aaaa cant' 'aaaa need'
 'aaaaa' 'aaaaa thanx' 'aaaaaaaa' 'aaaaaaaa hate' 'aaaaaaaaaaa'
 'aaaaaaaaaaa mcfli' 'aaaaaaaaaahhhhhhhh' 'aaaaaaaaaahhhhhhhh im'
 'aaaaaaaaaamaz' 'aaaaaaaaaamaz sd' 'aaaaaaaafternoon'
 'aaaaaaaafternoon delight']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 20 , max_depth=10)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=20)

In [ ]:
#Display the accuracy
from sklearn import metrics
y_pred = clf.predict(X_test)
metrics.accuracy_score(y_test, y_pred)

0.2495

Word Embedding Model

In [ ]:
tokenizer = Tokenizer(num_words= 5000)  # Limit to 5000 most frequent words
tokenizer.fit_on_texts(data['cleaned_content'])
X_sequences = tokenizer.texts_to_sequences(data['cleaned_content'])

In [ ]:
X_padded = pad_sequences(X_sequences, maxlen=100)

In [ ]:
y = data['sentiment'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.20, random_state=42)

RNN Model

In [ ]:
rnn = Sequential()
rnn.add(Embedding(input_dim=len(tokenizer.word_index) + 1,  # Number of words in the tokenizer's vocabulary
                    output_dim=100,  # Vector size for each word
                    input_length=100))  # Le

rnn.add(SimpleRNN(128,activation='relu',return_sequences=True))

rnn.add(SimpleRNN(64,activation='relu',return_sequences=False))

rnn.add(Dense(5, activation='softmax'))

# printing model summary
print(rnn.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_2 (SimpleRNN)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_3 (SimpleRNN)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(32000, 100) (8000, 100) (32000,) (8000,)


In [ ]:
X_padded.shape



(40000, 100)

In [ ]:
y.shape


(40000,)

In [ ]:
#Compile the model
rnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
print(X_padded.dtype)

int32


In [ ]:
print(y.dtype)

int64


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [ ]:
print(X_padded.shape)

(40000, 100)


In [ ]:
print(y.shape)

(40000,)


In [ ]:
valid_indices = [i for i in range(len(y)) if y[i] != 'empty']
X_padded = X_padded[valid_indices]
y = y[valid_indices]

LSTM Model

In [ ]:
max_words = 10000
max_len = 100

In [ ]:
model = Sequential(name="LSTM_Model")
model.add(Embedding(len(tokenizer.word_index)+1,
                    max_words,
                    input_length=max_words))

# adding a LSTM layer
model.add(LSTM(150, return_sequences=False))
model.add(BatchNormalization())  # Correct placement
model.add(Dropout(0.5))

# adding a dense layer with activation function of relu
model.add(Dense(50, activation='relu'))
model.add(BatchNormalization())  # Correct placement
model.add(Dropout(0.5))

# adding the final output activation with activation function of softmax
model.add(Dense(5, activation='softmax'))

# printing model summary
print(model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "LSTM_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
#Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_padded, y, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 279090000 elements. This may consume a large amount of memory.
  warnings.warn(


500/500 ━━━━━━━━━━━━━━━━━━━━ 133s 223ms/step - accuracy: 0.0033 - loss: nan - val_accuracy: 0.0015 - val_loss: nan
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 112s 223ms/step - accuracy: 0.0030 - loss: nan - val_accuracy: 0.0015 - val_loss: nan
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 109s 219ms/step - accuracy: 0.0031 - loss: nan - val_accuracy: 0.0015 - val_loss: nan
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 109s 219ms/step - accuracy: 0.0028 - loss: nan - val_accuracy: 0.0015 - val_loss: nan
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 112s 223ms/step - accuracy: 0.0029 - loss: nan - val_accuracy: 0.0015 - val_loss: nan


In [ ]:
#Display the accuracy
loss,accuracy = model.evaluate(X_padded, y)
print(f"Test Accuracy: {accuracy}")

1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.0037 - loss: nan
Test Accuracy: 0.002749999985098839


GRU Model

In [ ]:
model = Sequential()

# Adding an Embedding layer
model.add(Embedding(len(tokenizer.word_index) + 1,
                    max_words,
                    input_length=max_words))

# Adding a GRU layer instead of LSTM
model.add(GRU(150, return_sequences=False))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Adding a Dense layer with 'relu' activation
model.add(Dense(50, activation='relu'))
model.add(BatchNormalization())  # Correct placement for batch normalization
model.add(Dropout(0.5))

# Adding the output layer with 'softmax' activation for multi-class classification
model.add(Dense(5, activation='softmax'))

# Printing the model summary
print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
#Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_padded, y, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 124s 213ms/step - accuracy: 0.0033 - loss: nan - val_accuracy: 0.0015 - val_loss: nan
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 104s 207ms/step - accuracy: 0.0029 - loss: nan - val_accuracy: 0.0015 - val_loss: nan
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 207ms/step - accuracy: 0.0024 - loss: nan - val_accuracy: 0.0015 - val_loss: nan
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 207ms/step - accuracy: 0.0031 - loss: nan - val_accuracy: 0.0015 - val_loss: nan
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 207ms/step - accuracy: 0.0031 - loss: nan - val_accuracy: 0.0015 - val_loss: nan


In [ ]:
#Display the accuracy
loss,accuracy = model.evaluate(X_padded, y)
print(f"Test Accuracy: {accuracy}")

1250/1250 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - accuracy: 0.0037 - loss: nan
Test Accuracy: 0.002749999985098839
